# Анализ базы данных магазина электронных книг (SQL) 

**Цель проекта**: проанализировать базу данных магазина электронных книг, которая содержит информацию о книгах, издательствах, авторах, а также пользовательских обзорах книг. Эти данные помогут сформулировать ценностное предложение для новых продуктов магазина.

**Задачи**:

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, так мы исключим из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг среди книг с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Схема базы данных**:

[![image.png](https://i.postimg.cc/mrbZcWX2/image.png)](https://postimg.cc/WdHcKCcQ)

**Описание данных**:

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Загрузка данных

Загружаем библиотеки, подключаемся к бд:

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
books = pd.io.sql.read_sql('SELECT * FROM books', con = engine) 
authors = pd.io.sql.read_sql('SELECT * FROM authors', con = engine) 
publishers = pd.io.sql.read_sql('SELECT * FROM publishers', con = engine) 
ratings = pd.io.sql.read_sql('SELECT * FROM ratings', con = engine) 
reviews = pd.io.sql.read_sql('SELECT * FROM reviews', con = engine) 

Библиотеки и файлы загружены, далее ознакомимся с файлами.

Задаем функцию для исследования датасетов:

In [3]:
# функция для первичного изучения датафреймов:

def information_about_df (df: pd.DataFrame, limit_na_category: int = 10 ) -> None:
    print('Вывод шапки таблицы:')
    display(df.head())
    print('-'*40)
    print('Вывод общей информации:')
    print('')
    print(df.info())
    print('-'*40)
    print('Количество пропусков в данных:')
    print('')
    count = 0
    shape_0 = df.shape[0]
    for element in df.columns:
        if df[element].isna().sum()>0:
            print(element, ' - ', df[element].isna().sum(), 'пропусков, ', round(df[element].isna().sum()*100 / shape_0,2), '% от числа строк.')
            count = +1
        if count == 0:
            print(element,'- пропусков НЕТ')   
    print('-'*40)
    print('Количество дубликатов в данных:')
    print('')
    if df.duplicated().sum() >0:
        print('Дубликатов:', df.duplicated().sum())
    else:
        print('Дубликатов НЕ найдено')

### Файл "books"

Изучим файл "books":

In [4]:
information_about_df(books)

Вывод шапки таблицы:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


----------------------------------------
Вывод общей информации:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None
----------------------------------------
Количество пропусков в данных:

book_id - пропусков НЕТ
author_id - пропусков НЕТ
title - пропусков НЕТ
num_pages - пропусков НЕТ
publication_date - пропусков НЕТ
publisher_id - пропусков НЕТ
----------------------------------------
Количество дубликатов в данных:

Дубликатов НЕ найдено


Пропусков и дубликатов в данных нет.

 ### Файл "authors"

Изучим файл "authors":

In [5]:
information_about_df(authors)

Вывод шапки таблицы:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


----------------------------------------
Вывод общей информации:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None
----------------------------------------
Количество пропусков в данных:

author_id - пропусков НЕТ
author - пропусков НЕТ
----------------------------------------
Количество дубликатов в данных:

Дубликатов НЕ найдено


Пропусков и дубликатов в данных нет.

### Файл "publishers"

Изучим файл "publishers":

In [6]:
information_about_df(publishers)

Вывод шапки таблицы:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


----------------------------------------
Вывод общей информации:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None
----------------------------------------
Количество пропусков в данных:

publisher_id - пропусков НЕТ
publisher - пропусков НЕТ
----------------------------------------
Количество дубликатов в данных:

Дубликатов НЕ найдено


Пропусков и дубликатов в данных нет.

### Файл "ratings"

Изучим файл "ratings":

In [7]:
information_about_df(ratings)

Вывод шапки таблицы:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


----------------------------------------
Вывод общей информации:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None
----------------------------------------
Количество пропусков в данных:

rating_id - пропусков НЕТ
book_id - пропусков НЕТ
username - пропусков НЕТ
rating - пропусков НЕТ
----------------------------------------
Количество дубликатов в данных:

Дубликатов НЕ найдено


Пропусков и дубликатов в данных нет.

### Файл "reviews"

Изучим файл "reviews":

In [8]:
information_about_df(reviews)

Вывод шапки таблицы:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


----------------------------------------
Вывод общей информации:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None
----------------------------------------
Количество пропусков в данных:

review_id - пропусков НЕТ
book_id - пропусков НЕТ
username - пропусков НЕТ
text - пропусков НЕТ
----------------------------------------
Количество дубликатов в данных:

Дубликатов НЕ найдено


Пропусков и дубликатов в данных нет.

Переходим к задачам по ТЗ.

## Задача 1

Посчитаем, сколько книг вышло после 1 января 2000 года:

In [9]:
query = '''

SELECT COUNT(book_id) as book_count
FROM books
WHERE publication_date > '2000-01-01'

'''

pd.io.sql.read_sql(query, con = engine)

,book_count
0,819


После 1 января 2000 года вышло 819 книг.

## Задача 2

Для каждой книги посчитаем количество обзоров и среднюю оценку:

In [10]:
query = '''

SELECT b.book_id, 
       b.title, 
       COUNT(DISTINCT rv.review_id) AS reviews_count, 
       ROUND(AVG(r.rating),3) AS avg_rating
FROM books AS b

LEFT JOIN reviews AS rv
ON b.book_id = rv.book_id

LEFT JOIN  ratings AS r
ON b.book_id = r.book_id

GROUP BY b.book_id, b.title
ORDER BY reviews_count DESC

'''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,reviews_count,avg_rating
0,948,Twilight (Twilight #1),7,3.663
1,963,Water for Elephants,6,3.977
2,734,The Glass Castle,6,4.207
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.415
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.667
996,808,The Natural Way to Draw,0,3.000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000
998,221,Essential Tales and Poems,0,4.000


Средняя оценка для книг и количество обзоров посчитаны. 

## Задача 3

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц, так мы исключим из анализа брошюры:

In [11]:

query3 = '''SELECT 
publishers.publisher AS publisher, 
COUNT(books.book_id) AS books_leaders

FROM publishers 
INNER JOIN books ON books.publisher_id = publishers.publisher_id 

WHERE 
num_pages > 50

GROUP BY 
publishers.publisher 

ORDER BY 
books_leaders DESC 

LIMIT 1;
'''
pd.io.sql.read_sql(query3, con = engine)

,publisher,books_leaders
0,Penguin Books,42


Лидер - Пингвин Букс.

## Задача 4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками:

In [12]:
query = '''
SELECT
    authors.author AS author,
    ROUND(AVG(ratings.rating), 3) AS avg_rating
    FROM ratings
    JOIN books ON books.book_id = ratings.book_id
    JOIN authors ON books.author_id = authors.author_id
    WHERE ratings.book_id IN (
       SELECT ratings.book_id
       FROM ratings
       GROUP BY ratings.book_id
       HAVING COUNT(ratings.book_id) >= 50)
    GROUP BY authors.author_id
    ORDER BY AVG(ratings.rating) DESC
    LIMIT 1'''
data = pd.io.sql.read_sql(query, con = engine)
data

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287


Лидирует Джоан Роулинг.

## Задача 5

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок:

In [13]:
query5 = '''
SELECT
round(AVG(rr.reviews_amount), 2) AS avg_reviews

FROM 
(
SELECT 
ld.username AS username,
COUNT(reviews.review_id) AS reviews_amount 
        
FROM 
    (
    SELECT
    ratings.username AS username
    
    FROM
    ratings
    
    GROUP BY 
    ratings.username
    
    HAVING 
    COUNT(ratings.rating_id) > 50
    ) AS ld 
INNER JOIN reviews ON reviews.username = ld.username

GROUP BY
ld.username
) AS rr;

'''

pd.io.sql.read_sql(query5, con = engine)

,avg_reviews
0,24.33


Среднее количество обзоров от пользователей, которые поставили более 50 оценок - 24,3.

## Вывод по работе

В данном проекте были рассмотрены данные из датасетов с информацией по книгам, авторам, издательствам, рейтингам и отзывам пользователей. В ходе работы мы выяснили, что:

 - После 1 января 2000 года вышла 819 книг;
 - Для каждой книги посчитана средняя оценка и количество отзывов;
 - Определено издательство, которое выпустило наибольшее число книг толще 50 страниц - Пингвин букс;
 - Автор с самой высокой средней оценкой книг — Джоан Роулинг;
 - Среднее количество обзоров от пользователей, которые поставили более 50 оценок - 24,3.